# Machine learning for equipment failure
https://medium.com/swlh/machine-learning-for-equipment-failure-prediction-and-predictive-maintenance-pm-e72b1ce42da1

In [3]:
import sys
!{sys.executable} -m pip install xgboost

In [4]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import types
import pandas as pd

# def __iter__(self): return 0

## Import data

In [5]:
# Remove the data if you run this notebook more than once
# !rm equipment_failure_data_1.csv

# import first half fom github
!wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_1.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# Convert csv to pandas
pd_data_1 = pd.read_csv("equipment_failure_data_1.csv", sep=",", header=0)

# Remove the data if you run this notebook more than once
# !rm equipment_failure_data_2.csv

# Import the second half from github
# !wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_2.csv

# convert to pandas dataframe
pd_data_2 = pd.read_csv("equipment_failure_data_2.csv", sep=",", header=0)

# Concatenate the two data files into one dataframe
pd_data = pd.concat([pd_data_1, pd_data_2])

## Data exploration

In [7]:
pd_data.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


* ID - ID field that represents a specific machine.
* DATE - The date of the observation.
* REGION_CLUSTER - A field that represents the region in which the machine resides.
* MAINTENANCE_VENDOR - A field that represents the company that provides maintenance and service to the machine.
* MANUFACTURER - The company that manufactured the equipment in question.
* WELL_GROUP - A field representing the type of machine.
* EQUIPMENT_AGE - Age of the machine, in days.
* Sxx - Sensor values.
* EQUIPMENT_FAILURE - A '1' means that the equipment failed. A '0' means the equipment did not fail.

In [8]:
# Shape of the data
pd_data.shape

(307751, 16)

Num of machines: 421

In [ ]:
xxxx = pd